In [1]:
from func import *

import pickle as p
import glob 
import json
import subprocess
import getpass
import os
from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff, flip_axis
# import shutil

In [2]:
# ",".join(os.listdir('/home/abhijit/Jyotirmay/abdominal_segmentation/quickNAT_pytorch/create_datasets/datasets/lablmaps/KORA'))

In [3]:
dataset='KORA'
volume_txt_file = 'datasets/kora/volumes.txt'
data_dir = "/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti"
label_dir = 'datasets/lablmaps/KORA'

n4_corrected_data_dir = "temp/KORA/n4_corrected"
stictched_data_dir = "temp/KORA/stitched"
stitched_n4_corrected_data_dir = "temp/KORA/stitched_n4_corrected"

processed_path = 'temp/KORA/'
processed_dir = 'temp/KORA/processed'

one_time_n4_optimization = True
TARGET_RESOLUTION = [2,2,3]

In [4]:
# def rename_folder(dir_name,folder_id, prefix='KORA'):
#     create_if_not(f'{dir_name}/{prefix}{folder_id}')
#     os.rename(f'{dir_name}/{folder_id}/**', f'{dir_name}/{prefix}{folder_id}/**')

In [5]:
# for folder_path in os.listdir('/home/abhijit/Jyotirmay/abdominal_segmentation/quickNAT_pytorch/create_datasets/datasets/lablmaps/KORA'):
#     dir_name = '/'.join(folder_path.split('/')[:-1])
#     folder_id = folder_path.split('/')[-1]
#     rename_folder(dir_name, folder_id)

In [6]:
def load_kora_file_paths(load_from_txt_file=True):
    volumes_to_use = []
    if load_from_txt_file:
        with open(volume_txt_file) as file_handle:
                volumes_to_use = file_handle.read().splitlines()
    else:
        volumes_to_use = [name for name in os.listdir(data_dir)]
    
    file_paths = {}
    
    for vol in volumes_to_use:
        if vol == "":
            continue
            
        labelmap_paths = glob.glob(f'{label_dir}/{vol}/**')
#         vol = 'KORA'+vol if dataset=='KORA' else vol
        opp_paths = glob.glob(f'{data_dir}/{vol}/**opp_[0-9]**.nii.gz')
        in_paths = glob.glob(f'{data_dir}/{vol}/**in_[0-9]**.nii.gz')
        f_paths = glob.glob(f'{data_dir}/{vol}/**F_[0-9]**.nii.gz')
        w_paths = glob.glob(f'{data_dir}/{vol}/**W_[0-9]**.nii.gz')
        
        vol_madals_paths = dict(
        OPP=opp_paths,
        IN=in_paths,
        F=f_paths,
        W=w_paths
        )
        file_paths[str(vol)]=dict(
            VOLUME_PATHS=vol_madals_paths,
            LABEL_PATHS=labelmap_paths,
        )
    return file_paths
        

In [7]:
file_paths = load_kora_file_paths()

In [8]:
 print(json.dumps(file_paths,sort_keys=True, indent=4))

{
    "KORA2452879": {
        "LABEL_PATHS": [
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Pancreas.nrrd",
            "datasets/lablmaps/KORA/KORA2452879/Combi",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Liver.nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Kidney (left).nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Adrenal gland(right).nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Spleen.nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Gallbladder.nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-2_Kidney (right).nrrd",
            "datasets/lablmaps/KORA/KORA2452879/2452879_t1_vibe

In [9]:
# Individual RESCALING.
n4_dict = {}
for vol in file_paths.keys():
    if one_time_n4_optimization:
        break
    print(f'n4 processing part-1 started with {vol}...')
    n4_dict[vol] = []
    vol_parts = [[file, read_ras(file)] for file in file_paths[vol]['VOLUME_PATHS']['IN']]
    for orig_file, in_image in vol_parts:
        n4_dict[vol].append(rescale(in_image, vol, orig_file))

In [10]:
# TODO: EXECUTE 'sudo sh ./n4-anne.sh' from command line for n4 bais field generation of all in imgs.

for vol in file_paths.keys():
    if one_time_n4_optimization:
        break
    print(f'applying n4 bias field on opp scans of {vol}...')
    vol_parts = [[read_ras(opp_file), read_ras(n4_counterpart['IN_BIAS']), opp_file] for opp_file, n4_counterpart in zip(file_paths[vol]['VOLUME_PATHS']['OPP'],n4_dict[vol])]
    idx = 0
    file_paths[vol]['N4_1'] = []
    for opp_img, bias_field_img, opp_file in vol_parts:
        file_paths[vol]['N4_1'].append(apply_bias_field(opp_img, bias_field_img, opp_file, n4_dict[vol][idx], vol))
        idx+=1
#         n4_dict[vol].append(rescale(opp_image, vol, n4_dict[vol]))



In [11]:
# STITCHING VOL PARTS HERE
for vol in file_paths.keys():
    print(f'started with {vol}...')
    create_if_not(f'{n4_corrected_data_dir}/vol/{vol}')
    file_paths[vol]['ONE'] = {}
    for modality_key in file_paths[vol]['VOLUME_PATHS'].keys():
        print(f"processing {modality_key}")
        orig_modal_key = modality_key
        if one_time_n4_optimization:
            vol_parts = [read_ras(file) for file in file_paths[vol]['VOLUME_PATHS'][modality_key]]
        else:
            if modality_key == 'OPP':
                vol_parts = [read_ras(data_dict['OPP_CORRECTED']) for data_dict in file_paths[vol]['N4_1']]
                modality_key = modality_key+'_n4_corrected'
            else:
                vol_parts = [read_ras(file) for file in file_paths[vol]['VOLUME_PATHS'][modality_key]]

        ras_stitched = multi_vol_stitching(vol_parts)
        save_volume(ras_stitched, f'{n4_corrected_data_dir}/vol/{vol}/{modality_key}_ras_stitched')
        file_paths[vol]['ONE'][f'{orig_modal_key}'] = f'{n4_corrected_data_dir}/vol/{vol}/{modality_key}_ras_stitched.nii.gz'

started with KORA2456289...
processing OPP
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         235.26948547]
 [  0.           0.          -1.69999695 144.93167114]
 [  0.          -1.70138884   0.         -21.65872765]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          230.26948547]
 [   0.            0.           -1.69999695  144.93167114]
 [   0.           -1.66666663    0.         -256.65872192]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2456289
processing IN
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         235.26948547]
 [  0.           0.          -1.69999695 144.93167114]
 [  0.          

Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          240.02130127]
 [   0.            0.           -1.69999695  119.15551758]
 [   0.           -1.66666663    0.         -215.20669556]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2452879
processing W
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         245.02130127]
 [  0.           0.          -1.69999695 119.15551758]
 [  0.          -1.70138884   0.          19.79330063]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          240.02130127]
 [   0.            0.           -1.69999695  119.15551758]
 [   0.           -1

Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2453229
processing IN
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         262.03250122]
 [  0.           0.          -1.69999695 152.1910553 ]
 [  0.          -1.70138884   0.           0.52120578]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          257.03250122]
 [   0.            0.           -1.69999695  152.1910553 ]
 [   0.           -1.66666663    0.         -234.47879028]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2453229
processing F
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         2

Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          240.01109314]
 [   0.            0.           -1.69999695  116.75502014]
 [   0.           -1.66666663    0.         -283.12890625]
 [   0.            0.            0.            1.        ]], Image Shape: (288, 288, 160)
Transforming Images to RAS.....
0th img for stitching...
saving directory: temp/KORA/n4_corrected/vol/KORA2453290
started with KORA2460734...
processing OPP
Reading Files.....
Reading Nifti Files.....
Affine:[[ -1.70138884   0.           0.         242.58334351]
 [  0.           0.          -1.69999695 140.065979  ]
 [  0.          -1.70138884   0.          -9.54386711]
 [  0.           0.           0.           1.        ]], Image Shape: (288, 288, 176)
Transforming Images to RAS.....
Reading Files.....
Reading Nifti Files.....
Affine:[[  -1.66666663    0.            0.          237.58334351]
 [   0.            0.           -1.69999695  140.

KeyboardInterrupt: 

In [ ]:
# RESCALING INTENSITIES OF STITCHED VOLUME ABOVE 0
n4_dict = {}
for vol in file_paths.keys():
    print(f'n4 processing part-2 started with {vol}...')
    n4_dict[vol] = {}
    in_stitched_file_path, in_stitched_img = file_paths[vol]['ONE']['IN'], read_ras(file_paths[vol]['ONE']['IN'])
    n4_dict[vol]['N4_2'] = rescale(in_stitched_img, vol, in_stitched_file_path)

In [ ]:
# TODO: EXECUTE 'sudo sh ./n4-anne-2.sh' from command line for n4 bais field generation of all in imgs.
for vol in file_paths.keys():
    print(f'applying n4 bias field on stitched opp scans of {vol}...')

    opp_file = file_paths[vol]['ONE']['OPP']
    n4_counterpart = n4_dict[vol]['N4_2']
    print(opp_file, n4_counterpart['IN_BIAS'])
    opp_img = read_ras(opp_file)
    bias_field_img = read_ras(n4_counterpart['IN_BIAS'])

    file_paths[vol]['N4_2'] = apply_bias_field(opp_img, bias_field_img, opp_file, n4_dict[vol]['N4_2'], vol)

In [ ]:
with open('kora_file_paths.p', 'wb') as handle:
    p.dump(file_paths, handle, protocol=p.HIGHEST_PROTOCOL)

In [ ]:
with open('kora_file_paths.p', 'rb') as handle:
    file_paths = p.load(handle)

In [ ]:
def get_points_ext(label, vol):
    target_affine = vol.affine
    label_affine = label.affine
    labelmap2vol = npl.inv(target_affine).dot(label_affine)
    start_inv = np.floor(apply_affine(labelmap2vol, [0,0,0])).astype(np.int32)
    sx, sy,sz = start_inv
    end_inv = apply_affine(labelmap2vol, vol.shape).astype(np.int32)
    ex, ey, ez = end_inv
    return sx, sy,sz, ex, ey, ez

def affine_with_offsets(img):
    affine = img.affine.copy()
    img_h = img.header
    im_offx = img_h['qoffset_x']
    im_offy = img_h['qoffset_y']
    im_offz = img_h['qoffset_z']
    
    affine[3, :3] = [im_offx, im_offy, im_offz]
    return affine

def get_points_qoffset(label, vol):
    target_affine = affine_with_offsets(vol)
    label_affine = affine_with_offsets(label)
    
    labelmap2vol = npl.inv(target_affine).dot(label_affine)
    start_inv = np.floor(apply_affine(labelmap2vol, [0,0,0])).astype(np.int32)
    sx, sy,sz = start_inv
    end_inv = apply_affine(labelmap2vol, vol.shape).astype(np.int32)
    ex, ey, ez = end_inv
    return sx, sy,sz, ex, ey, ez


In [ ]:
def vol_label_fix(vol, label):
    vol_vol = np.product(vol.shape)
    label_vol = np.product(label.shape)
    
    world_shape = np.max(np.array([list(vol.shape), list(label.shape)]), axis=0)
    print(tuple(world_shape))
    final_label = np.zeros(tuple(world_shape))
    
    label_affine = label.affine
    vol_affine = vol.affine
#         final_label = np.zeros(vol.shape)
    target_affine = vol_affine
    target_header = vol.header
    target_dim_v = vol.shape

#         labelmap2vol = npl.inv(target_affine).dot(label_affine)
#         start_inv = np.floor(apply_affine(labelmap2vol, [0,0,0])).astype(np.int32)
#         sx, sy,sz = start_inv
#         end_inv = apply_affine(labelmap2vol, target_dim_v).astype(np.int32)
#         final_label = np.zeros(end_inv)
#         ex, ey, ez = end_inv
#         print("seg start inv v: ",start_inv , "segm end inv v:",end_inv)
    sx,sy,sz,ex,ey,ez = get_points(label, vol)
    print('points:',sx,sy,sz,ex,ey,ez)
    sx,sy,sz,ex,ey,ez = np.abs([sx,sy,sz,ex,ey,ez])
#     sx,sy,sz,ex,ey,ez = get_points_ext(label, vol)
#     print('points_ext:', sx,sy,sz,ex,ey,ez)
#     sx,sy,sz,ex,ey,ez = get_points_qoffset(label, vol)
#     print('points_qoffset:', sx,sy,sz,ex,ey,ez)
    final_label[0:ex+sx, 0:ey+sy, 0:ez+sz] = label.get_fdata()
    final_label = np.flip(final_label, axis=0)
# #         final_label = np.flip(final_label, axis=1)
    final_label = np.flip(final_label, axis=2)
    final_label_img = nb.Nifti1Image(final_label, target_affine, target_header)
    volume, label = vol, final_label_img
#     else:
# #         final_label = np.zeros(label.shape)
#         target_affine = label_affine
#         target_header = label.header
#         target_dim_v = label.shape
        
# #         labelmap2vol = npl.inv(target_affine).dot(vol_affine)
# #         start_inv = np.floor(apply_affine(labelmap2vol, [0,0,0])).astype(np.int32)
# #         sx, sy,sz = start_inv
# #         end_inv = apply_affine(labelmap2vol, target_dim_v).astype(np.int32)
# #         final_label = np.zeros(end_inv)
# #         ex, ey, ez = end_inv
# #         print("seg start inv v: ",start_inv , "segm end inv v:",end_inv)
#         sx,sy,sz,ex,ey,ez = np.abs(get_points(vol, label))
#         print(sx,sy,sz,ex,ey,ez, get_points(vol, label))
# #         final_label = np.flip(final_label, axis=0)
# #         final_label = np.flip(final_label, axis=1)
# #         final_label = np.flip(final_label, axis=2)
#         final_label[0:ex+sx, 0:ey+sy, 0:ez+sz] = vol.get_fdata()
        
#         final_label_img = nb.Nifti1Image(final_label, target_affine, target_header)
#         volume, label = final_label_img, label
    
    return volume, label

def label_parts(label_parts):
    stitched_label = None
    reference_labelmap = nb.load(f'temp/ref_img.nii.gz')
    mode = 'constant'
    label_shape = np.max([img.shape for img, _, _ in label_parts], axis=0)
    print('final_label_stitching shape:',label_shape)
    stitched_label = np.zeros(label_shape)
    for labelmap_img, lidx, lname in label_parts:
        print('lp:bfr:', lidx, lname, labelmap_img.shape, np.unique(labelmap_img.get_fdata()))
        labelmap_img = makeit_3d(labelmap_img)
        if reference_labelmap is None:
            reference_labelmap = labelmap_img
#         else:
        labelmap_img = resample_from_to(labelmap_img, [label_shape, reference_labelmap.affine], order=3, mode=mode, cval=0)
        labelmap_img = labels_integerify(labelmap_img)
        
        print(np.unique(labelmap_img.get_fdata()), labelmap_img.shape)
        
#         sx,sy,sz,ex,ey,ez = get_points(labelmap_img, reference_labelmap)
#         sx,sy,sz,ex,ey,ez = np.abs([sx,sy,sz,ex,ey,ez])
#         print(sx,sy,sz,ex,ey,ez)
        
        labelmap = labelmap_img.get_fdata()
        labelmap = np.multiply(lidx, labelmap)
        x, y, z = labelmap.shape
#         stitched_label[0:ex+sx, 0:ey+sy, 0:ez+sz] += labelmap
        stitched_label[:x, :y, :z] += labelmap
#         if stitched_label is None:
#             stitched_label = labelmap
#         else:
#             print(stitched_label.shape)
#             stitched_label = np.add(stitched_label, labelmap)
        print("###############################################################################################") 
        
    labelmap = np.round(stitched_label)
    empty_header = nb.Nifti1Header()
    reference_labelmap.header['dim'][1:4] = label_shape
    print(reference_labelmap.header)
    stitched_labeled_img = nb.Nifti1Image(labelmap, reference_labelmap.affine, reference_labelmap.header)
    
    return stitched_labeled_img

In [ ]:
# vol = nb.load(f'{processed_dir}/volume/KORA2453666.nii.gz')
# vol.header['dim'][1:4] = [20,20,30]
# print(vol.header['dim'][1:4])
# print(vol.header)

# data = np.zeros((400, 400, 600))
# affine = np.diag([2,2,3,1])
# empty_header = nibabel.Nifti1Header()
# r_img = nb.Nifti1Image(data, affine, empty_header)
# save_volume(r_img, f'temp/ref_img')

In [ ]:
print("STARTING KORA LABEL-MAPS.")
print('Reading Label Maps.....')
for vol in file_paths.keys():
    print(vol)
#     if vol not in ['KORA2460830']:
#         continue
    later = None
    print(file_paths[vol]['LABEL_PATHS'])
    if len(file_paths[vol]['LABEL_PATHS']) == 0:
        print(f"#################### ALERT:: NO LABELPATHS IN THE DICTIONARY FOR {vol} #########################")
        continue
    volume = nb.load(file_paths[vol]['N4_2']['OPP_CORRECTED'])
    img_ras_list = []
    for label_file_to_read in file_paths[vol]['LABEL_PATHS']:
        img_ras, lidx, labelname = read_ras(label_file_to_read, is_label=True)
        if img_ras is None:
            continue
        print(img_ras.shape, np.unique(img_ras.get_fdata()))
        img_ras = makeit_3d(img_ras)
        print('After 3d confirmed:', img_ras.shape, np.unique(img_ras.get_fdata()))
        mode='constant'
        img_ras = resample_to_output(img_ras, TARGET_RESOLUTION, order=3, mode=mode, cval=0)
        img_ras = labels_integerify(img_ras)
        print(np.unique(img_ras.get_fdata()))
    #     img_ras = resample_from_to(img_ras, [volume.shape, img_ras.affine])
        if labelname == 'PANCREAS':
            later = [img_ras, lidx, labelname]
        else:
            img_ras_list.append([img_ras, lidx, labelname])
    if later is not None:
        img_ras_list.append(later)
    
    s_label = label_parts(img_ras_list)
    volume, s_label = vol_label_fix(volume, s_label)
    #     volume, stitched_label = combine_nako_seg(file_paths[vol]['LABEL_PATHS'],'' ,file_paths[vol]['N4_2']['OPP_CORRECTED'])

    print('Viewing Stitched Images.....')
    volume_3_view_viewer(get_volume_data(volume))
    volume_3_view_viewer(get_volume_data(s_label))

    print('Saving Processed & Stitched Image.....')
    save_volume(volume, f'{processed_dir}/volume/{vol}')
    save_volume(s_label, f'{processed_dir}/label/{vol}')
    
print('FINISHED.')

In [ ]:
points: -2 -148 -250 243 0 -5
points_ext: 3 -149 237 249 1 476

In [ ]:
# from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff, flip_axis
# vol = nb.load(f'{processed_dir}/volume/KORA2453666.nii.gz')
# label = nb.load(f'{processed_dir}/label/KORA2453666.nii.gz')

# label_d = flip_axis(label.get_fdata(), axis=0)
# label_d = flip_axis(label_d, axis=2)
# label = nb.Nifti1Image(label_d, label.affine, label.header)

# print('Viewing Stitched Images.....')
# volume_3_view_viewer(get_volume_data(vol))
# volume_3_view_viewer(get_volume_data(label))

# # save_volume(volume, f'{processed_dir}/volume/KORA2453666')
# save_volume(label, f'{processed_dir}/label/KORA2453666')

In [ ]:
def crop_roi(paths):
    affine = None
    data = np.zeros((256,256,400))
    for path in paths:
        print(path)
        img = nb.load(path)
        if affine is None:
            affine = img.affine
#         img = resample_from_to(img, [(300, 300, 300), img.affine])
        img_data= img.get_fdata()
        x,y,z = img.shape
        print(x,y,z)
        data[:x, :y, :z] += img_data
        
    empty_header = nibabel.Nifti1Header()
    all_label_img = nb.Nifti1Image(data, affine, empty_header)
    return all_label_img

def crop(paths, shape):
    s1, e1, s2, e2, s3, e3 = shape
    for path in paths:
        print(path)
        img = nb.load(path)
#         img = resample_from_to(img, [(300, 300, 300), img.affine])
        img_data= img.get_fdata()
        data = img_data[s1:e1, s2:e2, s3:e3]
        img = nb.Nifti1Image(data, img.affine, img.header)
        save_path = '/'.join(path.split('/')[:-1])
        vol_id = path.split('/')[-1].split('.')[0]
        save_volume(img, f'{save_path}_cropped/{vol_id}')

def remove_black(labelmap):
    clean_labels = []
    start, end = None, None
    for i, frame in enumerate(labelmap):
        unique, counts = np.unique(frame, return_counts=True)
        if counts[0] / sum(counts) < .99:
            clean_labels.append(frame)
            if start is None:
                start = i
            end = None
        if end is None:
            end = i
    if end is None:
        end = labelmap.shape[0]
            
    return np.array(clean_labels), start, end




In [ ]:
processed_dir = 'temp/KORA/processed'
kora_labels = glob.glob(f'{processed_dir}/label/**')
kora_volumes = glob.glob(f'{processed_dir}/volume/**')
processed_dir = 'temp/NAKO/processed'
nako_labels = glob.glob(f'{processed_dir}/label/**')
nako_volumes = glob.glob(f'{processed_dir}/volume/**')
processed_dir = 'temp/UKB/processed'
ukb_labels = glob.glob(f'{processed_dir}/label/**')
ukb_volumes = glob.glob(f'{processed_dir}/volume/**')

all_labels = kora_labels + nako_labels + ukb_labels
all_volumes = kora_volumes + nako_volumes + ukb_volumes

print(all_labels, all_volumes)


In [ ]:
def resize(img, shape=(256, 256, 400)):
    img = resample_from_to(img, [shape, img.affine])
    return img

In [ ]:
for vol in all_volumes:
    vol_id = vol.split('/')[-1].split('.')[0]
    save_path = '/'.join(vol.split('/')[:-2])
    print(vol_id, save_path)
    vol_img = nb.load(vol)
    resized_img = resize(vol_img)
    save_volume(resized_img, f'{save_path}_resized/volume/{vol_id}')

In [ ]:
for vol in all_labels:
    vol_id = vol.split('/')[-1].split('.')[0]
    save_path = '/'.join(vol.split('/')[:-2])
    print(vol_id, save_path)
    vol_img = nb.load(vol)
    resized_img = resize(vol_img)
    save_volume(resized_img, f'{save_path}_resized/label/{vol_id}')

In [ ]:
processed_dir = 'temp/KORA/processed_resized'
kora_labels = glob.glob(f'{processed_dir}/label/**')
kora_volumes = glob.glob(f'{processed_dir}/volume/**')
processed_dir = 'temp/NAKO/processed_resized'
nako_labels = glob.glob(f'{processed_dir}/label/**')
nako_volumes = glob.glob(f'{processed_dir}/volume/**')
processed_dir = 'temp/UKB/processed_resized'
ukb_labels = glob.glob(f'{processed_dir}/label/**')
ukb_volumes = glob.glob(f'{processed_dir}/volume/**')

In [ ]:
kls = crop_roi(kora_labels)
volume_3_view_viewer(get_volume_data(kls))
save_volume(kls, f'all_labels_merged/all_kora_labels')

kls = crop_roi(nako_labels)
volume_3_view_viewer(get_volume_data(kls))
save_volume(kls, f'all_labels_merged/all_nako_labels')

kls = crop_roi(ukb_labels)
volume_3_view_viewer(get_volume_data(kls))
save_volume(kls, f'all_labels_merged/all_ukb_labels')

In [ ]:
img = nb.load(f'all_labels_merged/all_kora_labels.nii.gz')
img_data = img.get_fdata()
img_d1, s1,e1 = remove_black(img_data)
img_data1 = np.moveaxis(img_data, 1, 0)
img_d2, s2,e2 = remove_black(img_data1)
img_data2 = np.moveaxis(img_data, 2, 0)
img_d3,s3,e3 = remove_black(img_data2)
# img_data = np.moveaxis(img_data, axis=0)

print('kora', img_d1.shape, img_d2.shape, img_d3.shape, s1,e1,s2,e2,s3,e3)

img = nb.load(f'all_labels_merged/all_nako_labels.nii.gz')
img_data = img.get_fdata()
img_d1,s1,e1 = remove_black(img_data)
img_data1 = np.moveaxis(img_data, 1, 0)
img_d2,s2,e2 = remove_black(img_data1)
img_data2 = np.moveaxis(img_data, 2, 0)
img_d3,s3,e3 = remove_black(img_data2)
# img_data = np.moveaxis(img_data, axis=0)

print('nako', img_d1.shape, img_d2.shape, img_d3.shape, s1,e1,s2,e2,s3,e3)

img = nb.load(f'all_labels_merged/all_ukb_labels.nii.gz')
img_data = img.get_fdata()
img_d1,s1,e1 = remove_black(img_data)
img_data1 = np.moveaxis(img_data, 1, 0)
img_d2,s2,e2 = remove_black(img_data1)
img_data2 = np.moveaxis(img_data, 2, 0)
img_d3,s3,e3 = remove_black(img_data2)
# img_data = np.moveaxis(img_data, axis=0)

print('ukb', img_d1.shape, img_d2.shape, img_d3.shape, s1,e1, s2,e2, s3,e3)

In [ ]:
# KORA = 224,133,179   ---  0, 223, 6, 138, 60, 238
# NAKO = 208,149,96    --- 7 214 21 169 156 251
# UKB= 166, 118, 97    --- 40 209 42 159 217 313

# KORA sample vol size = 250, 188, 370
# NAKO sample vol size = 226,184,316
# UKB sample vol size = 246, 137, 239
# sample target shape = 256, 256,400

In [ ]:
crop(kora_volumes, [0, 223, 6, 138, 60, 238])

In [ ]:
crop(kora_labels, [0, 223, 6, 138, 60, 238])

In [ ]:
# file_dict= 
# {
#     VOL_ID: {
#         VOLUME_PATHS: {
#             OPP: [],
#             IN: [],
#             F: [],
#             W: []from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff, flip_axis
#         },
#         LABEL_PATHS: [],
#         N4_1: [
#             {
#                 SCALED: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled.nii.gz',
#                 IN_BIAS: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled_bias_field.nii.gz',
#                 IN_CORRECTED: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled_corrected.nii.gz',
#                 OPP_CORRECTED: '',
#                 MIN: u_min,
#                 MAX: u_max
#             },
#             {},
#             {}
#         ],from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff, flip_axis
#         ONE     : {
#             OPP: '',
#             IN: '',
#             F: '',
#             W: '',
#         },
#         N4_2: {
#             SCALED: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled.nii.gz',
#             IN_BIAS: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled_bias_field.nii.gz',
#             IN_CORRECTED: f'{n4_corrected_data_dir}/vol/{vol_id}/{new_filename}_n4_scaled_corrected.nii.gz',
#             OPP_CORRECTED: '',
#             MIN: u_min,
#             MAX: u_max
#         },
        
#     }
# }